In [1]:
from kaggle_secrets import UserSecretsClient
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
import requests

In [2]:
# Function to make predictions
def make_predictions(model, texts):
    # Tokenize the input texts
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')

    # Move inputs to the same device as the model
    device = model.device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted labels
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predictions = torch.argmax(logits, dim=1).tolist()
    confidence_scores = probabilities.max(dim=1).values.tolist()

    return predictions, confidence_scores


In [3]:
# Function to load the model
def load_model(model_path):
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [4]:
user_secrets = UserSecretsClient()

# Connect to supabase
supabase_url = "https://fglqovplibiyttjzqxuj.supabase.co"
supabase_key = user_secrets.get_secret("SUPABASE_KEY")

# Make a query
query_url = f"{supabase_url}/rest/v1/currentversions?select=maintext&limit=1000"
headers = {"apikey": supabase_key}
response = requests.get(query_url, headers=headers)
data = response.json()


# Extract main text from the response
main_texts = [row["maintext"] for row in data]

# Preprocess text as needed

# Load the model
model_path = '/kaggle/input/nfrd/pytorch/v1.0.0/1/model.pth'
model = load_model(model_path)

# Self labeling process
self_labeled_batch = []
for text in main_texts:
    predictions, confidence_scores = make_predictions(model, [text])
    labeled_text = {
        "text": text,
        "prediction": predictions[0],
        "confidence score": confidence_scores[0]
    }
    self_labeled_batch.append(labeled_text)



config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
print(self_labeled_batch)

[{'text': 'Informationen zur Anzeige:\nAnzeigeninhalt:\nBergisch Gladbach\nIngenieurin / Ingenieur (m/w/d) oder Baustofftechnologin / Baustofftechnologe (m/w/d) (Master/Uni-Diplom)\nSeit der Römerzeit ist die Verwendung von zement\xadgebundenen Stoffen in der Verkehrsinfrastruktur unerlässlich. Jedoch steht der Betonbau mit Blick auf den erforderlichen Beitrag zur Erreichung der Klimaschutzziele vor großen Herausforderungen. Gleichzeitig kommt der Sicherstellung der Verfügbarkeit der vorhandenen Brückenbauwerke im Straßennetz nicht zuletzt auch vor dem Hintergrund eines nachhaltigen Betriebs eine hohe Bedeutung zu. Damit der Baustoff Beton in den nächsten 100 Jahren seine Leistung bringen kann, muss er nachhaltiger werden. Den aktuellen Herausforderungen stehen technische Entwicklungen auf dem Gebiet der digitalen Technologien gegenüber, die zur Weiterentwicklung in der Planung, Ausschreibung, Durchführung, Dokumentation und Qualitätssicherung von baulichen Maßnahmen beitragen können. 